In [1]:
import pickle
from universal.algos import *
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from universal import tools
from universal import result
from itertools import chain
from torch.distributions import Categorical
from env.Portfolio_management import CustomEnv
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from PPOdemo import PPO
import seaborn as sns

E:\anaconda\envs\protfoilo\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [46]:
df = pd.read_csv('30stocks.csv')

In [47]:
df_original_train = df.head(round(0.8*len(df)))

In [48]:
df_original_train

,AAPL,JCI,JBHT,IVZ,CAH,CF,CERN,CELG,CDNS,CCL,...,BA,BXP,BWA,BSX,BRK.B,BLL,BLK,BK,BIIB,BF.B
0,67.8542,32.2094,69.07,27.31,45.18,45.232,44.050,50.065,14.16,39.01,...,76.56,105.14,37.470,7.70,97.25,22.680,238.16,27.90,164.44,32.540
1,68.5614,32.1780,68.32,27.43,45.22,45.124,43.750,49.970,14.22,38.72,...,75.87,105.29,37.465,7.63,97.13,22.570,239.45,27.93,163.96,32.050
2,66.8428,32.4712,67.13,27.71,45.54,44.068,43.775,49.480,14.36,39.02,...,75.99,107.03,37.705,7.59,97.70,22.445,238.86,27.90,165.45,32.100
3,66.7156,32.9843,66.92,27.63,45.44,43.578,43.810,49.535,14.25,37.46,...,74.78,107.18,37.585,7.45,97.97,22.600,242.52,28.17,165.91,32.055
4,66.6556,33.3717,67.41,27.78,46.00,43.572,43.950,49.755,14.20,37.35,...,74.93,107.35,37.570,7.53,99.21,22.605,245.57,28.10,164.59,32.165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1002,128.7500,42.5000,98.32,29.59,75.35,36.630,53.860,117.410,25.82,55.47,...,163.97,129.27,41.030,23.96,163.61,38.265,373.00,45.02,281.69,45.010
1003,128.5300,42.3000,97.24,29.55,74.44,35.630,53.620,115.540,28.93,55.31,...,162.26,130.47,40.470,25.10,163.00,36.660,371.64,44.80,264.23,45.140
1004,129.0800,42.6600,99.66,30.90,74.87,35.680,53.660,115.640,29.13,55.28,...,162.40,130.72,40.600,25.22,163.94,36.920,378.00,45.67,264.67,45.700
1005,130.2900,42.3000,99.52,30.46,75.97,35.490,54.050,116.090,29.38,55.50,...,163.98,131.32,40.570,25.14,163.42,36.470,377.42,45.66,264.91,45.450


In [50]:
cov = df_original_train.cov().to_numpy()

In [51]:
cov

array([[ 4.50780618e+02,  4.06408151e+01,  9.62193171e+01,
         3.21545815e+01,  2.53660254e+02,  6.99980593e+01,
         1.16606850e+02,  4.19216274e+02,  6.11833562e+01,
         1.09604043e+02,  1.07483947e+02,  1.89599632e+02,
         2.22514974e+01,  1.62496762e+02,  8.48373951e+01,
         1.45166220e+01, -4.00875029e+01,  5.70144857e+01,
         3.03369260e+02,  1.96004911e+02,  1.61927111e+01,
         6.95637431e+01,  2.79553453e+02,  1.05387213e+02,
         6.85246215e+02,  9.89127485e+01,  9.16660736e+02,
         9.59294699e+01],
       [ 4.06408151e+01,  1.99189028e+01,  1.36176333e+01,
         8.74541843e+00,  2.73175263e+01,  7.78364588e+00,
         1.36250196e+01,  3.33999603e+01,  5.91421738e+00,
         3.75032730e+00,  1.22535519e+01,  2.71523149e+01,
         1.79595950e+01,  1.57998928e+01,  7.47764759e+00,
         6.34163262e+00,  2.84632726e+01, -8.83224546e-01,
         4.13896002e+01,  2.83986810e+01,  2.05290702e+01,
         7.99272686e+00,  3.95

In [52]:
for i in range(5):
    print(i)

0
1
2
3
4
